###Задание 1. Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации


- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов


- сравнить все реализованные методы, сделать выводы


In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
# import re
import numpy as np
# from nltk.corpus import stopwords
# from tqdm.notebook import tqdm
# from multiprocessing import Pool
# import pymorphy2
# import matplotlib.pyplot as plt
import pyconll
from nltk.tag import DefaultTagger, UnigramTagger, BigramTagger, TrigramTagger

In [3]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, classification_report
from sklearn.preprocessing import LabelEncoder

In [4]:
!wget -O /content/sample_data/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-train.conllu
!wget -O /content/sample_data/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-dev.conllu

--2022-10-26 09:49:39--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17229332 (16M) [text/plain]
Saving to: ‘/content/sample_data/ru_syntagrus-ud-train.conllu’

/content/sample_dat 100%[===================>]  16.43M  --.-KB/s    in 0.1s    

2022-10-26 09:49:39 (136 MB/s) - ‘/content/sample_data/ru_syntagrus-ud-train.conllu’ saved [17229332/17229332]

--2022-10-26 09:49:40--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-Taiga/master/ru_taiga-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubuser

In [5]:
full_train = pyconll.load_from_file('/content/sample_data/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('/content/sample_data/ru_syntagrus-ud-dev.conllu')

In [11]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Снова ADV
приобрел VERB
дозу NOUN
, PUNCT

В ADP
женщине NOUN
важна ADJ
верность NOUN
, PUNCT
а CCONJ
не PART
красота NOUN
. PUNCT



In [12]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])

In [15]:
fdata_train[:2]

[[('Снова', 'ADV'), ('приобрел', 'VERB'), ('дозу', 'NOUN'), (',', 'PUNCT')],
 [('В', 'ADP'),
  ('женщине', 'NOUN'),
  ('важна', 'ADJ'),
  ('верность', 'NOUN'),
  (',', 'PUNCT'),
  ('а', 'CCONJ'),
  ('не', 'PART'),
  ('красота', 'NOUN'),
  ('.', 'PUNCT')]]

In [16]:
fdata_sent_test[:2]

[['🤘', 'Хей', '!'], ['Ставь', '❤️']]

In [36]:
taggers_scores = {'unigram_tagger': [],
                  'bigram_tagger': [], 
                  'bi_unigram_tagger': [],
                  'trigram_tagger': [],
                  'tri_unigram_tagger': [],
                  'tri_bi_unigram_tagger': []}

In [37]:
unigram_tagger = UnigramTagger(fdata_train)
taggers_scores['unigram_tagger']=unigram_tagger.evaluate(fdata_test)

bigram_tagger = BigramTagger(fdata_train)
taggers_scores['bigram_tagger']=bigram_tagger.evaluate(fdata_test)

bi_unigram_tagger = BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))
taggers_scores['bi_unigram_tagger']=bi_unigram_tagger.evaluate(fdata_test)

trigram_tagger = TrigramTagger(fdata_train)
taggers_scores['trigram_tagger']=trigram_tagger.evaluate(fdata_test)

tri_unigram_tagger = TrigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))
taggers_scores['tri_unigram_tagger']=tri_unigram_tagger.evaluate(fdata_test)

tri_bi_unigram_tagger = TrigramTagger(fdata_train,
                                     backoff=BigramTagger(fdata_train,
                                                          backoff=UnigramTagger(fdata_train)))
taggers_scores['tri_bi_unigram_tagger']=tri_bi_unigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: DeprecationWarning: 
  Function evaluate() has been deprecated.  

In [44]:
pd.DataFrame(taggers_scores, index=['score']).T.sort_values('score', ascending=False)

,score
tri_bi_unigram_tagger,0.686708
tri_unigram_tagger,0.686510
bi_unigram_tagger,0.685915
unigram_tagger,0.683142
bigram_tagger,0.135994
trigram_tagger,0.096177


In [47]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [78]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

test_enc_labels = le.transform(test_label)

In [80]:
scores = {'vectorizer': [],
          'f1_score': [],
          'accuracy': []}

In [79]:
analyzers = ['word', 'char']
n_feats = [1000, 5000, 10000, 15000]
vectorizers = [CountVectorizer, TfidfVectorizer, HashingVectorizer]

In [82]:
def prepare_fit_predict(vec):
    X_train = vec.fit_transform(train_tok)
    X_test = vec.transform(test_tok)
    lr = LogisticRegression(random_state=0, max_iter=100)
    lr.fit(X_train, train_enc_labels)
    return lr.predict(X_test)

In [83]:
for vect_cls in vectorizers:
    for analyzer in analyzers:
        if vect_cls == HashingVectorizer:
            for n_feat in n_feats:
                vectorizer = vect_cls(ngram_range=(1, 3), analyzer=analyzer, n_features=n_feat)
                scores['vectorizer'].append(vectorizer)
                pred = prepare_fit_predict(vectorizer)
                scores['f1_score'].append(f1_score(test_enc_labels, pred, average='weighted'))
                scores['accuracy'].append(accuracy_score(test_enc_labels, pred))
                
        else:
            vectorizer = vect_cls(ngram_range=(1, 3), analyzer=analyzer)
            scores['vectorizer'].append(vectorizer)
            pred = prepare_fit_predict(vectorizer)
            scores['f1_score'].append(f1_score(test_enc_labels, pred, average='weighted'))
            scores['accuracy'].append(accuracy_score(test_enc_labels, pred)) 

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [87]:
pd.set_option('max_colwidth', -1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [88]:
pd.DataFrame(scores).sort_values('f1_score', ascending=False)

,vectorizer,f1_score,accuracy
1,"CountVectorizer(analyzer='char', ngram_range=(1, 3))",0.860490,0.866977
3,"TfidfVectorizer(analyzer='char', ngram_range=(1, 3))",0.837610,0.849544
10,"HashingVectorizer(analyzer='char', n_features=10000, ngram_range=(1, 3))",0.826014,0.839045
11,"HashingVectorizer(analyzer='char', n_features=15000, ngram_range=(1, 3))",0.824430,0.838847
9,"HashingVectorizer(analyzer='char', n_features=5000, ngram_range=(1, 3))",0.820969,0.834093
8,"HashingVectorizer(analyzer='char', n_features=1000, ngram_range=(1, 3))",0.783539,0.799525
0,"CountVectorizer(ngram_range=(1, 3))",0.543662,0.508023
2,"TfidfVectorizer(ngram_range=(1, 3))",0.536882,0.502377
7,"HashingVectorizer(n_features=15000, ngram_range=(1, 3))",0.524233,0.508716
6,"HashingVectorizer(n_features=10000, ngram_range=(1, 3))",0.516544,0.506537


###Задание 2. Проверить, насколько хорошо работает NER

Данные брать из http://www.labinform.ru/pub/named_entities/

- проверить NER из nltk/spacy/deeppavlov.

- написать свой NER, попробовать разные подходы.

    - передаём в сетку токен и его соседей.

    - передаём в сетку только токен.

    - свой вариант.

- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [4]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-10-27 08:41:34--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip’

collection5.zip     100%[===================>]   1.81M   928KB/s    in 2.0s    

2022-10-27 08:41:37 (928 KB/s) - ‘collection5.zip’ saved [1899530/1899530]



In [5]:
import zipfile
with zipfile.ZipFile('/content/collection5.zip', 'r') as zip_ref:
    zip_ref.extractall('/content/')

In [17]:
!pip install corus

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 83 kB 1.5 MB/s 


In [24]:
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [50]:
import nltk
import corus
from corus import load_ne5
from corus import load_rudrec
from razdel import tokenize

In [58]:
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.


True

In [19]:
path = '/content/Collection5'
records = load_ne5(path)

In [29]:
next(records)

Ne5Markup(
    id='last_47',
    text='\r\nШанцев скорректировал структуру нижегородского правительства и назначил себе новых заместителей\r\n\r\nГубернатор Валерий Шанцев внес кадровые изменения в состав правительства Нижегородской области. Как сообщает Шанцев в своем блоге, он уже подписал новую структуру правительства области.\r\n\r\n"В нынешнем виде, за небольшими исключениями, она существует 3 года. За это время многое изменилось, появились новые вызовы - экономические, социальные. Убежден, что правительство должно оперативно и профессионально реагировать на них, должно идти в ногу со временем. В течение полугода я проводил консультации. Для повышения эффективности правительства в целом и каждого отдельного блока в частности считаю целесообразными следующие изменения.", - пишет губернатор.\r\n\r\n"Блок экономики и финансов. Задачами Владимира Иванова (вице-губернатор Нижегородской области) вижу планирование и формирование бюджета, экономическое планирование. С целью развития и мод

####**NLTK**

In [48]:
document = next(load_ne5(path)).text

In [60]:
document

'Генеральным директором "Мечел-Энерго" назначен Ю.Ямпольский.\r\n\r\n06.03.2012, Москва 19:41:19 Генеральным директором ООО "Мечел-Энерго" (входит в группу "Мечел") назначен Юрий Ямпольский, который сменил на этом посту исполняющего обязанности генерального директора Анатолия Чернакова. Об этом говорится в сообщении компании. Новый глава компании будет управлять генерирующими и энергосбытовыми предприятиями, координировать работу подразделений, обеспечивать их эффективность.\r\n\r\nКак отмечается в сообщении, Ю.Ямпольский имеет многолетний практический опыт в сфере энергетики, с ноября 2010г. и до настоящего времени он занимал должность заместителя генерального директора по экономике и финансам, первого заместителя генерального директора, заместителя генерального директора ОАО "Научно-исследовательский инжиниринговый центр межрегиональных распределительных сетевых компаний", с 1997г. занимал руководящие должности.\r\n\r\n"Мечел" объединяет производителей угля, железорудного концентрата

In [59]:
{(' '.join(c[0] for c in chunk), chunk.label() ) for chunk in nltk.ne_chunk(nltk.pos_tag(nltk.word_tokenize(document))) if hasattr(chunk, 'label') }

{('ADR', 'ORGANIZATION'),
 ('Генеральным', 'GPE'),
 ('ОАО', 'ORGANIZATION'),
 ('Основным', 'ORGANIZATION'),
 ('РСБУ', 'ORGANIZATION')}

####**SPACY**

In [61]:
!pip install -U spacy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [66]:
!python -m spacy download ru_core_news_sm

2022-10-27 11:39:54.909385: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 15.3 MB 4.3 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_sm')


In [63]:
import spacy
from spacy import displacy

In [67]:
nlp = spacy.load('ru_core_news_sm')

In [68]:
ny_bb = document
article = nlp(ny_bb)
displacy.render(article, jupyter=True, style='ent')

In [87]:
article

Генеральным директором "Мечел-Энерго" назначен Ю.Ямпольский.

06.03.2012, Москва 19:41:19 Генеральным директором ООО "Мечел-Энерго" (входит в группу "Мечел") назначен Юрий Ямпольский, который сменил на этом посту исполняющего обязанности генерального директора Анатолия Чернакова. Об этом говорится в сообщении компании. Новый глава компании будет управлять генерирующими и энергосбытовыми предприятиями, координировать работу подразделений, обеспечивать их эффективность.

Как отмечается в сообщении, Ю.Ямпольский имеет многолетний практический опыт в сфере энергетики, с ноября 2010г. и до настоящего времени он занимал должность заместителя генерального директора по экономике и финансам, первого заместителя генерального директора, заместителя генерального директора ОАО "Научно-исследовательский инжиниринговый центр межрегиональных распределительных сетевых компаний", с 1997г. занимал руководящие должности.

"Мечел" объединяет производителей угля, железорудного концентрата, стали, проката, ф

####**TF NER**

In [72]:
words_docs = []
for ix, rec in enumerate(load_ne5(path)):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [73]:
df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])

In [74]:
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [76]:
df_words.head(10)

,word,tag
0,Генеральным,OUT
1,директором,OUT
2,"""",OUT
3,Мечел-Энерго,ORG
4,"""",OUT
5,назначен,OUT
6,Ю,PER
7,.,PER
8,Ямпольский,PER
9,.,OUT


In [77]:
import tensorflow as tf

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D, GlobalMaxPooling1D, Conv1D, GRU, LSTM, Dropout, Input
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization

In [78]:
from sklearn import model_selection, preprocessing, linear_model

train_x, valid_x, train_y, valid_y = model_selection.train_test_split(df_words['word'], df_words['tag'])

# labelEncode целевую переменную
encoder = preprocessing.LabelEncoder()
train_y = encoder.fit_transform(train_y)
valid_y = encoder.fit_transform(valid_y)

In [79]:
train_data = tf.data.Dataset.from_tensor_slices((train_x, train_y))
valid_data = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))

train_data = train_data.batch(16)
valid_data = valid_data.batch(16)

In [80]:
AUTOTUNE = tf.data.AUTOTUNE

train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
valid_data = valid_data.cache().prefetch(buffer_size=AUTOTUNE)

In [81]:
def custom_standardization(input_data):
    return input_data

vocab_size = 30000
seq_len = 10

vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    #ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [90]:
len(vectorize_layer.get_vocabulary())

29927

In [83]:
embedding_dim = 64

class modelNER(tf.keras.Model):
    def __init__(self):
        super(modelNER, self).__init__()
        self.emb = Embedding(vocab_size, embedding_dim)
        self.gPool = GlobalMaxPooling1D()
        self.fc1 = Dense(300, activation='relu')
        self.fc2 = Dense(50, activation='relu')
        self.fc3 = Dense(6, activation='softmax')

    def call(self, x):
        x = vectorize_layer(x)
        x = self.emb(x)
        pool_x = self.gPool(x)
        
        fc_x = self.fc1(pool_x)
        fc_x = self.fc2(fc_x)
        
        concat_x = tf.concat([pool_x, fc_x], axis=1)
        prob = self.fc3(concat_x)
        return prob

In [84]:
mmodel = modelNER()

In [85]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [86]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 302s 24ms/step - loss: 0.2960 - accuracy: 0.9137 - val_loss: 0.2138 - val_accuracy: 0.9370
Epoch 2/3
12444/12444 [==============================] - 297s 24ms/step - loss: 0.1265 - accuracy: 0.9625 - val_loss: 0.2222 - val_accuracy: 0.9400
Epoch 3/3
12444/12444 [==============================] - 307s 25ms/step - loss: 0.1100 - accuracy: 0.9653 - val_loss: 0.2746 - val_accuracy: 0.9402


In [93]:
pred = mmodel.predict(valid_data)

4148/4148 [==============================] - 7s 2ms/step


In [120]:
pred_y = np.array([np.argmax(x) for x in pred])

In [121]:
report = classification_report(valid_y, np.array(pred_y))
print(report)

              precision    recall  f1-score   support

           0       0.89      0.89      0.89      1086
           1       0.86      0.79      0.83      1189
           2       0.92      0.77      0.84       615
           3       0.88      0.55      0.68      3407
           4       0.94      0.99      0.97     54793
           5       0.99      0.69      0.82      5278

    accuracy                           0.94     66368
   macro avg       0.91      0.78      0.84     66368
weighted avg       0.94      0.94      0.94     66368



In [122]:
vectorize_layer = TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    ngrams=(1, 3),
    output_sequence_length=seq_len)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_data = train_data.map(lambda x, y: x)
vectorize_layer.adapt(text_data)

In [123]:
mmodel = modelNER()

In [124]:
mmodel.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

In [125]:
mmodel.fit(train_data, validation_data=valid_data, epochs=3)

Epoch 1/3
12444/12444 [==============================] - 306s 24ms/step - loss: 0.2929 - accuracy: 0.9146 - val_loss: 0.2121 - val_accuracy: 0.9367
Epoch 2/3
12444/12444 [==============================] - 301s 24ms/step - loss: 0.1252 - accuracy: 0.9629 - val_loss: 0.2299 - val_accuracy: 0.9397
Epoch 3/3
12444/12444 [==============================] - 298s 24ms/step - loss: 0.1096 - accuracy: 0.9654 - val_loss: 0.2570 - val_accuracy: 0.9398


In [126]:
pred = mmodel.predict(valid_data)
pred_y = np.array([np.argmax(x) for x in pred])
report = classification_report(valid_y, np.array(pred_y))
print(report)

4148/4148 [==============================] - 7s 2ms/step
              precision    recall  f1-score   support

           0       0.88      0.88      0.88      1086
           1       0.86      0.79      0.82      1189
           2       0.91      0.77      0.83       615
           3       0.88      0.55      0.68      3407
           4       0.94      0.99      0.97     54793
           5       0.99      0.69      0.81      5278

    accuracy                           0.94     66368
   macro avg       0.91      0.78      0.83     66368
weighted avg       0.94      0.94      0.93     66368

